In [1]:
import cv2, numpy as np, os, radialProfile, glob, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import griddata
from sklearn.svm import SVC
from create_features import ezdata

In [4]:
test_files = [
    "data/test/deepfake",   # class 0
    "data/test/face2face",  # class 1
    "data/test/faceswap",   # class 2
    "data/test/original"    # class 3
]

pickle_out = "data/emsemble_test_2k.pkl"

ezdata(2000, test_files, pickle_out, epsilon=10)

# Load in classifiers
IMPORTANT! The index number matches the class number when the multiclass SVM was trained.

In [2]:
classifiers = []

# loads classifiers - deepfake [0]
pkl_temp = open("data/pickled_classifiers/deepfakeSVM_ep500_C6.37_gam0.86.pkl", 'rb')
temp = pickle.load(pkl_temp)
classifiers.append(temp)
pkl_temp.close()

# loads classifiers - face2face [1]
pkl_temp = open("data/pickled_classifiers/face2faceSVM_ep500_C6.37_gam0.86.pkl", 'rb')
temp = pickle.load(pkl_temp)
classifiers.append(temp)
pkl_temp.close()

# loads classifiers - faceswap [2]
pkl_temp = open("data/pickled_classifiers/faceswapSVM_ep500_C6.37_gam0.86.pkl", 'rb')
temp = pickle.load(pkl_temp)
classifiers.append(temp)
pkl_temp.close()

# loads in the multiclass classifier
pkl_temp = open("data/pickled_classifiers/multiclass_SVM_C6.37_gam0.86.pkl", 'rb')
multiSVM = pickle.load(pkl_temp)
pkl_temp.close()

# Load in Testing Data

In [11]:
pkl_temp = open("data/emsemble_test_2k.pkl", 'rb')
data = pickle.load(pkl_temp)
pkl_temp.close()
X = data["data"]
Y = data["label"]

# Classification: generating seperate predictions initially

In [12]:
multi_weights = multiSVM.predict_proba(X)
pickle_file = open("data/final_testing_matrices/multiclass_weights.pkl", 'wb')
pickle.dump(multi_weights, pickle_file)
pickle_file.close()

In [30]:
# binary classifiers
# 0: deepfake
# 1: face2face
# 2: faceswap
import random
pred = []
for i in range(len(classifiers)):
    tempClassify = classifiers[i].predict(X)
    pred.append(tempClassify)
pred = np.transpose(pred)
# 0 = fake
# 1 = real
pickle_file = open("data/final_testing_matrices/binary_preds.pkl", 'wb')
pickle.dump(pred, pickle_file)
pickle_file.close()

# Augmenting individual preds with weight

In [31]:
pkl_temp = open("data/final_testing_matrices/multiclass_weights.pkl", 'rb')
multiclass_weights = pickle.load(pkl_temp)
pkl_temp.close()

pkl_temp = open("data/final_testing_matrices/binary_preds.pkl", 'rb')
binary_preds = pickle.load(pkl_temp)
pkl_temp.close()

In [36]:
final_preds = []
for i in range(binary_preds.shape[0]):
    pred_f = 0
    for j in range(binary_preds.shape[1]):
        pred_f += (binary_preds[i][j] * multiclass_weights[i][j])
    # if < 0.5, fake.
    final_preds.append(round(pred_f))
final_preds = np.transpose(final_preds)

# Scoring
## Checking how many true positives (correctly found fakes) there were

In [50]:
fakeList = final_preds[:6000].tolist()
truePos = fakeList.count(0)
print((truePos)/6000)

0.8785


## False negatives.

In [56]:
falseNeg = fakeList.count(1)
print((falseNeg)/6000)

0.12566666666666668
